<a href="https://colab.research.google.com/github/anibohara2000/hand-syn/blob/master/handwriting_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import files
# files.upload()

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding
import numpy as np

In [0]:
letters = np.load('letters.npy', allow_pickle=True).item()
def coord_to_offset(d):
    for key in d:
        newl = []
        for l in d[key]:
            for i in range(1, len(l)):
                l[i] = [l[i][0] - l[i - 1][0], l[i][1] - l[i - 1][1], l[i][2]]
            l[0] = [0, 0, l[0][2]]
            newl.append(l)
        d[key] = newl
    return d
letters = coord_to_offset(letters)

In [0]:
# Number of words to consider as features
# max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 200

# # Load IMDB sentiment data. Skip top most common words.
# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features,
#                                                       skip_top=10)

# # Reverse sequences
# x_train = [x[::-1] for x in x_train]
# x_test = [x[::-1] for x in x_test]


In [0]:
# Pad sequences
for l in letters:
    letters[l] = sequence.pad_sequences(letters[l], maxlen=maxlen, padding='post')
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [0]:
x = []
y = []
for key in letters:
    for l in letters[key]:
        x.append(key)
        y.append(l)
x_train = np.zeros((len(x), maxlen, 52))
y_train = np.zeros((len(y), maxlen, 3))
for i in range(len(x)):
    y_train[i, :, :] = y[i]
    l = [0.0 for k in range(52)]
    if(ord(x[i])-65 <= 25):
      l[ord(x[i])-65] = 1.0
    else:
      l[26+ord(x[i])-97] = 1.0
    x_train[i, :,:] = np.array([np.array(l) for j in range(maxlen)])
    
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)

In [7]:
print(x_train.shape)
print(y_train.shape)

(1144, 200, 52)
(1144, 200, 3)


In [8]:
print(len(letters['a'][0]))

200


In [0]:
def make_model(batch_size=None):
  source = Input(shape=(maxlen, 52), batch_size=batch_size, dtype=tf.float32, name='Input')
  lstm = LSTM(3, name = 'LSTM', return_sequences=True)(source)
  model = tf.keras.Model(inputs=[source], outputs=[lstm])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='mse',
      metrics=['acc'])
  return model


In [10]:
tf.keras.backend.clear_session()
training_model = make_model(batch_size = 128)
training_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 200, 52)            0         
_________________________________________________________________
LSTM (LSTM)                  (128, 200, 3)             672       
Total params: 672
Trainable params: 672
Non-trainable params: 0
_________________________________________________________________


In [11]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.1.60.218:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8661455894642400886)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5216825019016262898)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3481956981224669402)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, 

In [12]:
import time
start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=40,
                    batch_size=128 * 8,
                    validation_split=0.1)
print("--- %s seconds ---" % (time.time() - start_time))

tpu_model.save_weights('./tpu_model.h5', overwrite=True)

Train on 1144 samples, validate on 115 samples
Epoch 1/40
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(128,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(128, 200, 52), dtype=tf.float32, name='Input_10'), TensorSpec(shape=(128, 200, 3), dtype=tf.float32, name='LSTM_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 4.029510974884033 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(15,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(15, 200, 52), dtype=tf.float32, name='Input_10'), TensorSpec(shape=(15, 200, 3), dtype=tf.float32, name='LSTM_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder fo

In [13]:
inferencing_model = make_model(batch_size=None)
inferencing_model.load_weights('./tpu_model.h5')
inferencing_model.evaluate(x_test, y_test, batch_size=128 * 8)

NameError: ignored